In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('dictionary.csv')
del df['No.']

In [3]:
df

,English,Toki Pona
0,I,mi
1,you (singular),sina
2,he,ona
3,we,mi (mute)
4,you (plural),sina (mute)
...,...,...
202,with,"kepeken, lon poka"
203,and,"en, li, e"
204,if,la
205,because,tan


In [4]:
syllables = ['0', '1', '2']
space = ' '

In [5]:
# Define a function to convert to any base n
def convert_to_base_n(x, base):
    if x == 0:
        return '0'

    base_str = ''
    while x > 0:
        remainder = x % base
        base_str = str(remainder) + base_str
        x //= base
    
    return base_str

# Specify the base you want (e.g., base 3)
base_n = len(syllables)

# Add a new column with base-n indices
df[f'Base-{base_n} Index'] = df.index.map(lambda x: convert_to_base_n(x, base_n))

In [23]:
print(df.to_string())

                    English                      Toki Pona Base-3 Index
0                         I                             mi            0
1            you (singular)                           sina            1
2                        he                            ona            2
3                        we                      mi (mute)           10
4              you (plural)                    sina (mute)           11
5                      they                     ona (mute)           12
6                      this                             ni           20
7                      that                             ni           21
8                      here                         lon ni           22
9                     there                         lon ni          100
10                      who                       jan seme          101
11                     what                           seme          102
12                    where                        ma seme      

In [7]:
import numpy as np
from scipy.io.wavfile import write

# Define a function to generate a sine wave for a given frequency and duration
def generate_sine_wave(frequency, duration_ms, amplitude=1.0, sample_rate=44100):
    t = np.linspace(0, duration_ms / 1000, int(sample_rate * (duration_ms / 1000)), endpoint=False)
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave

# Define the number of notes in an octave and their frequencies (adjust as needed)
num_notes = len(syllables)
base_frequency = 440.0  # A4, the 4th A on a piano
octave = [base_frequency * 2**(i/12) for i in range(num_notes)]

# Define the duration of each note in milliseconds (adjust as needed)
note_duration_ms = 500

# Generate and save audio files for each note
for note_index, frequency in enumerate(octave):
    note_wave = generate_sine_wave(frequency, note_duration_ms)
    write(f'note_{note_index}.wav', 44100, note_wave.astype(np.float32))

In [8]:
!pip3 install pydub

In [24]:
from pydub import AudioSegment
import numpy as np

def generate_silence(duration_ms, sample_width=2):
    # Generate silence using numpy
    sample_rate = 44100  # Adjust as needed
    num_samples = int(sample_rate * duration_ms / 1000)
    silence = np.zeros(num_samples, dtype=np.int16)
    
    # Convert the numpy array to an AudioSegment with the specified sample width
    audio_segment = AudioSegment(
        silence.tobytes(),
        frame_rate=sample_rate,
        sample_width=sample_width,
        channels=1
    )
    
    return audio_segment

def generate_sine_wave(frequency, duration_ms, sample_width=2):
    # Generate a sine wave using numpy
    sample_rate = 44100  # Adjust as needed
    t = np.linspace(0, duration_ms / 1000, int(sample_rate * duration_ms / 1000), endpoint=False)
    sine_wave = 32767 * np.sin(2 * np.pi * frequency * t)
    
    # Convert the numpy array to an AudioSegment with the specified sample width
    audio_segment = AudioSegment(
        sine_wave.tobytes(),
        frame_rate=sample_rate,
        sample_width=sample_width,
        channels=1
    )
    
    return audio_segment

def create_sound_from_base_n(base_n_string, n, output_file):
    # Define the frequencies for each note in an octave
    octave_frequencies = [440 * (2 ** (i / n)) for i in range(n)]
    
    # Create a mapping from digits to frequencies
    digit_to_frequency = {str(i): octave_frequencies[i] for i in range(n)}
    
    # Define the sample width (2 bytes for 16-bit audio)
    sample_width = 4
    
    # Create a list of audio segments for each digit in the base-n string
    audio_segments = []
    for digit in base_n_string:
        if digit == ' ':
            # Pause for the duration of a note
            audio_segments.append(generate_silence(duration_ms=100, sample_width=sample_width))  # 500 milliseconds pause
        else:
            # Play the note corresponding to the digit
            frequency = digit_to_frequency.get(digit, 0)  # Use 0 Hz for unsupported digits
            audio_segment = generate_sine_wave(frequency, duration_ms=100, sample_width=sample_width)  # 500 milliseconds per note
            audio_segments.append(audio_segment)
    
    # Concatenate the audio segments
    final_audio = sum(audio_segments)
    
    # Export the audio as a sound file
    final_audio.export(output_file, format="wav")

# Example usage
base_n_string = "10 20211"
n = 3
output_file = "output.wav"
create_sound_from_base_n(base_n_string, n, output_file)
